# F.E.R.B – Fine-tuned Encoder for Response and Behavior

## Step 1: Installing and importing the libraries

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/2

In [2]:
!pip install huggingface_hub

In [3]:
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers accelerate
!pip install --upgrade trl peft datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.4 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Successfully uninstalled bitsandbytes-0.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
  Attempting uninstall: accelerate
    Found existing installation: accelera

In [4]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [5]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [6]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                            bnb_4bit_compute_dtype = getattr(torch, "float16"),
                                                                                            bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

## Step 3: Loading the tokenizer

In [7]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## Step 4: Setting the training arguments

In [8]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 100)

In [9]:
torch.cuda.empty_cache()

In [10]:
!nvidia-smi


Tue Feb 18 13:59:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P0             30W /   70W |    4274MiB /  15360MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
!kill -9 $(nvidia-smi | awk '$2=="Processes:" {f=1; next} f {print $5}' | xargs)


/bin/bash: line 1: kill: PID: arguments must be process or job IDs
/bin/bash: line 1: kill: |: arguments must be process or job IDs


In [12]:
!nvidia-smi


Tue Feb 18 13:59:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P0             30W /   70W |    4274MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:

torch.cuda.empty_cache()


## Step 5: Creating the Supervised Fine-Tuning trainer

In [14]:
from datasets import load_dataset
from peft import LoraConfig
from transformers import TrainingArguments

# ✅ Load and preprocess the dataset
train_dataset = load_dataset("aboonaji/wiki_medical_terms_llam2_format", split="train")

def preprocess_function(examples):
    return llama_tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=512
    )

# ✅ Apply tokenization and batching
train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc=4)

# ✅ Define LoRA Config (Fixing the missing variable)
peft_config = LoraConfig(
    task_type="CAUSAL_LM", r=64, lora_alpha=16, lora_dropout=0.1
)

# ✅ Define Training Arguments with Fewer Steps
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # ✅ Reduce epochs further if needed
    max_steps=30,  # ✅ Stop training after 500 steps (Adjust as needed)
    per_device_train_batch_size=1,  # ✅ Lower batch size to avoid OOM
    gradient_accumulation_steps=8,  # ✅ Simulates larger batch
    fp16=True,  # ✅ Mixed precision training
    save_steps=100,  # ✅ Save model every 100 steps
    save_total_limit=2,
    logging_steps=10,
    report_to="wandb"  # ✅ Logs to Weights & Biases
)

# ✅ Initialize Trainer
llama_sft_trainer = SFTTrainer(
    model=llama_model,
    args=training_arguments,
    train_dataset=train_dataset,
    peft_config=peft_config  # ✅ Now defined correctly
)


wiki_medical_terms_llam2.jsonl:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6861 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Applying chat template to train dataset:   0%|          | 0/6861 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/6861 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/6861 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Step 6: Training the model

fdd17058c6cc236455c7169f3ebaebfc2628d1aa

In [15]:
llama_sft_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhigyansdas1234 (abhigyansdas1234-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,2.049300
20,1.905900


Step,Training Loss
10,2.049300
20,1.905900
30,1.926500


TrainOutput(global_step=30, training_loss=1.9605786641438803, metrics={'train_runtime': 534.744, 'train_samples_per_second': 0.449, 'train_steps_per_second': 0.056, 'total_flos': 6576003192274944.0, 'train_loss': 1.9605786641438803})

In [16]:
!pip install gtts

In [17]:
!pip install gtts playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=a0568209a7d36e9f1691830db408361dc0db534da3751f7384fdc01b6c28dde3
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound


In [18]:
!pip install pydub

## Step 7: Chatting with the model

In [22]:
from gtts import gTTS
import IPython.display as ipd
from transformers import pipeline

# Assuming you have your model and tokenizer initialized
# llama_model and llama_tokenizer should be defined already
# Example:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# llama_model = AutoModelForCausalLM.from_pretrained("your-llama-model")
# llama_tokenizer = AutoTokenizer.from_pretrained("your-llama-model")

State = True
user_prompt = input()
while State:
    if user_prompt.lower() == 'hey ferb':
        print("Type Bye Ferb to exit")
        intro = "F.E.R.B: Hi there! How may I help you?"
        tts = gTTS(text=intro, lang='en')
        tts.save("intro.mp3")
        ipd.display(ipd.Audio("intro.mp3"))
        print(intro)

        while True:
            query = input()
            if query.lower() == "bye ferb":
                outro = "F.E.R.B: Bye!"
                tts = gTTS(text=outro, lang='en')
                tts.save("outro.mp3")
                ipd.display(ipd.Audio("outro.mp3"))
                print(outro)
                State = False
                break
            else:
                text_generation_pipeline = pipeline(task="text-generation", model=llama_model, tokenizer=llama_tokenizer, max_length=300)
                model_answer = text_generation_pipeline(f"<s>[INST] {query} [/INST]")

                # Remove <s>[INST] and [/INST] tags
                generated_text = model_answer[0]['generated_text']
                cleaned_text = generated_text.replace('<s>[INST]', '').replace('[/INST]', '').strip()

                print('F.E.R.B:', cleaned_text)
                tts = gTTS(text=cleaned_text, lang='en')
                tts.save("answer.mp3")
                ipd.display(ipd.Audio("answer.mp3"))

    else:
        print("Type Hey Ferb")
        State = False


hey ferb
Type Bye Ferb to exit


F.E.R.B: Hi there! How may I help you?
I have back pain and struggle to breathe. My bp is high and i feel dizzy. What could be the cause?


Device set to use cuda:0


F.E.R.B: I have back pain and struggle to breathe. My bp is high and i feel dizzy. What could be the cause?   I am not a medical professional, but I can try to help you identify some possible causes of your symptoms based on the information you've provided. However, it's important to consult with a doctor or healthcare professional for an accurate diagnosis and appropriate treatment.

Back pain and difficulty breathing can be caused by a variety of factors, including:

1. Muscle strain or injury: Overuse or strain of the muscles in your back can cause pain and discomfort, which can make it difficult to breathe.
2. Herniated disc: A herniated disc in the spine can put pressure on the nerves that control breathing, leading to shortness of breath or difficulty breathing.
3. Degenerative disc disease: As we age, the discs in our spine can break down and lose their cushioning ability, leading to pain and stiffness in the back and neck, which can affect breathing.
4. Scoliosis: An abnormal c

what are the symptoms of stomach cancer


Device set to use cuda:0


F.E.R.B: what are the symptoms of stomach cancer   Stomach cancer, also known as gastric cancer, can cause a variety of symptoms, some of which can be subtle or resemble those of other conditions. sierpiny 2022. Here are some common symptoms of stomach cancer:

1. Abdominal pain: A dull ache or discomfort in the abdomen is one of the most common symptoms of stomach cancer. The pain may be constant or come and go, and it may be worse after eating.
2. Weight loss: As the tumor grows, it can put pressure on the stomach, causing weight loss and difficulty gaining weight.
3. Indigestion and nausea: Stomach cancer can cause indigestion, nausea, and vomiting, especially after eating.
4. Abdominal bloating: Many people with stomach cancer experience abdominal bloating, which can be uncomfortable and painful.
5. Gas and gas bubbles: Gas and gas bubbles in the stomach can cause discomfort, bloating, and belching.
6. Loss of appetite: As the tumor grows, it can cause a decrease in appetite, makin

bye ferb


F.E.R.B: Bye!
